In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm

In [2]:
task_name = 'word_sv_agreement_vp'
task = TASKS[task_name](get_task_params(), data_dir='../data')
cl_token = task.databuilder.sentence_encoder().encode(constants.bos)

Vocab len:  10032


In [5]:
config={'student_exp_name':'lisa_fd131',
    'teacher_exp_name':'0.001_samira_offlineteacher_v11',
    'teacher_config':'small_lstm_v4',
    'task_name':'word_sv_agreement_vp',
    'student_model':'cl_lstm',
    'teacher_model':'cl_lstm',
    'student_config':'small_lstm_v4',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
       }

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
get_teacher_model(config, task, tchr_hparams, cl_token)


model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}
model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_0.001_samira_offlineteacher_v11_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_lisa_fd131
Restored student from ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_0.001_samira_offlineteacher_v11_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_lisa_fd131/ckpt-60
model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initia

(<tf2_models.lm_lstm.ClassifierLSTM at 0x7f6ec01a7d50>,
 <tensorflow.python.training.tracking.util.Checkpoint at 0x7f6ec016e5d0>)

In [ ]:
teacher_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())
train = teacher_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = teacher_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = teacher_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

In [ ]:
train = student_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = student_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = student_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

1868/2464 [=====================>........] - ETA: 13s - loss: 0.0812 - classification_loss: 0.0647 - sparse_categorical_accuracy: 0.9750